#### Initially, we import the libraries used throughout. Besides the common ones we make use of bqplot interactive visualizations, widgets can be used with ipywidgets and datetime for dates and time values mentioned.

In [1]:

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import bqplot
import matplotlib.pyplot as plt
import ipywidgets
import datetime as dt
import numpy as np


In C:\Users\12172\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\12172\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\12172\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\12172\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases lat

In [2]:
buildings = pd.read_csv('building_inventory.csv') # Reading CSV 
buildings_data = buildings[buildings['Year Acquired'] != 0] 
buildings_data = buildings_data[buildings_data['Year Constructed'] != 0] 
# Removing Year Acquired/Year Constructed values that are 0

#buildings_data = buildings[(buildings['Year Acquired'] != 0) & (buildings['Year Constructed'] != 0) \
#& (buildings['Square Footage'] != 0) ]---- can't do this because we need square footage values

In [3]:
## checking if NAN
buildings_data.isna().sum()

Agency Name                  0
Location Name                0
Address                     51
City                         0
Zip code                     0
County                      24
Congress Dist                0
Congressional Full Name    154
Rep Dist                     0
Rep Full Name               22
Senate Dist                  0
Senator Full Name           22
Bldg Status                  0
Year Acquired                0
Year Constructed             0
Square Footage               0
Total Floors                 0
Floors Above Grade           0
Floors Below Grade           0
Usage Description            0
Usage Description 2         19
Usage Description 3         49
dtype: int64

### Part 1:

### converting the vlaues to array to make it easy to used in heat_map Plotting: 

In [4]:
agency_array = np.array(buildings_data['Agency Name'])
district_array = np.array(buildings_data['Congress Dist'])

agency_array
district_array

array([17, 17, 17, ..., 12, 12, 16], dtype=int64)

#### Grouping the building inventory's data by 'Congress Dist' as well as 'Agency Name' to obtain the sum of 'Square Footage'


In [5]:
Total_sqfootage = buildings_data.groupby(['Congress Dist','Agency Name'])['Square Footage'].sum().reset_index()
Total_sqfootage

#print(Total_sqfootage.to_string())

Total_sqfootage.shape

(153, 3)

The pivot function reshapes the building inventory DataFrame organized by given index. Converting the table into an array to use the indices after we link both the visualizations to change values accordingly. 

In [6]:
buildings_pivot = Total_sqfootage.pivot_table(index='Congress Dist', columns='Agency Name',values=['Square Footage'])
buildings_pivot

Square Footage  \
Agency Name   Appellate Court / Fifth District   
Congress Dist                                    
0                                          NaN   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
5                                          NaN   
6                                          NaN   
7                                          NaN   
8                                          NaN   
9                                          NaN   
10                                         NaN   
11                                         NaN   
12                                     15124.0   
13                                         NaN   
14                                         NaN   
15                                         NaN   
16                                         NaN   
17                                         NaN   
18                                         NaN   

                                                 \
Agency Name   Appellate Court / Fourth District   
Congress Dist                                     
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   
5                                           NaN   
6                                           NaN   
7                                           NaN   
8                                           NaN   
9                                           NaN   
10                                          NaN   
11                                          NaN   
12                                          NaN   
13                                      16400.0   
14                                          NaN   
15                                          NaN   
16                                          NaN   
17                                          NaN   
18                                          NaN   

                                                 \
Agency Name   Appellate Court / Second District   
Congress Dist                                     
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   
5                                           NaN   
6                                           NaN   
7                                           NaN   
8                                       43330.0   
9                                           NaN   
10                                          NaN   
11                                          NaN   
12                                          NaN   
13                                          NaN   
14                                          NaN   
15                                          NaN   
16                                          NaN   
17                                          NaN   
18                                          NaN   

                                                                         \
Agency Name   Appellate Court / Third District Chicago State University   
Congress Dist                                                             
0                                          NaN                      NaN   
1                                          NaN                1191992.0   
2                                          NaN                      NaN   
3                                          NaN                      NaN   
4                                          NaN                      NaN   
5                                          NaN                      NaN   
6         

In [7]:
buildings_values  = buildings_pivot.values

#buildings_values

In [8]:

color_sc = bqplot.ColorScale(scheme = 'BrBG',
                            min=np.nanmin(buildings_pivot), 
                            max=np.nanmax(buildings_pivot))
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# create axis - for colors, x & y
c_ax = bqplot.ColorAxis(scale = color_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc, label = "Agency Names", label_offset="50px")
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', label = "Congressional Districts", label_offset="50px")


heat_map = bqplot.GridHeatMap(color = buildings_pivot,
                              scales = {'color': color_sc,
                                        'row': y_sc,
                                        'column':x_sc },
                              interactions = {'click': 'select'}, # make interactive on click of each box
                              anchor_style = {'fill':'blue'}, # to make our selection blue
                              selected_style = {'opacity': 1.0}, # make 100% opaque if box is selected
                              unselected_style = {'opacity': 0.8}) # make a little see-through if not

# stiring and combing into 1 figure
fig = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax])

fig

Figure(axes=[ColorAxis(orientation='vertical', scale=ColorScale(max=14693427.0, min=2364.0, scheme='BrBG'), si…

Here, we have plotted the heat_map with customized values. however, we have yet not defined the output values after every click It just shows the changes in values after clicks on gridmap. These change may vary from colour schemes, achor style and type of interaction. The nanmax and nanmin fucntion of the numpy array will return maximum and minumum values ignoring NAN values, thus plotting all the values within the heatmap colors and all NAN values in black color. The color axis shows the density/intensity of the values within the heat_map.

## Part 2 : 

In [9]:
build_year_sqfootage  = buildings_data.groupby('Year Acquired')['Square Footage'].sum().reset_index()

build_year= np.array(build_year_sqfootage['Year Acquired'].astype('int'))
build_sqfootage = np.array(build_year_sqfootage['Square Footage'].astype('int'))

#build_sqfootage

#build_year
#build_sqfootage

In [10]:
##Sclaes:
x_scl = bqplot.LinearScale(min=1753,max=2019) # note: for dates on x-axis
y_scl = bqplot.LinearScale(min=500, max=5000000)
width_line = bqplot.LinearScale()
#x_scl = bqplot.LinearScale()
#y_scl = bqplot.LinearScale()



##Axes:
ax_xcl = bqplot.Axis(label='Year Acquired', scale=x_scl,label_offset="50px")
ax_ycl = bqplot.Axis(label='Total Square Footage', scale=y_scl, orientation='vertical', side='left',label_offset="50px")

##Plot_line_graph:
Plot_lines = bqplot.Lines(x = build_year,y = build_sqfootage, scales={'x':x_scl, 'y':y_scl,
                            'width': width_line}, marker_size=15, marker = 'circle', colors = ['DarkOrange'], 
                             opacities = [0.7], line_style = 'solid',stroke_width = 2 )

## Figure:
fig_1 = bqplot.Figure(marks = [Plot_lines], axes = [ax_xcl, ax_ycl])
fig_1

Figure(axes=[Axis(label='Year Acquired', label_offset='50px', scale=LinearScale(max=2019.0, min=1753.0)), Axis…

Here, we are plotting Year_acquired vs total Square Footage and grouping by Year acquired.converting the year acquired and square footage to int to use it later. For total square footage we can use logscale to change the visibility of the scale. 

### Combining part 1 and part 2: for interactivity: 

In [11]:


color_sc = bqplot.ColorScale(scheme = 'BrBG',
                            min=np.nanmin(buildings_pivot), 
                            max=np.nanmax(buildings_pivot))
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# create axis - for colors, x & y
c_ax = bqplot.ColorAxis(scale = color_sc, 
                        orientation = 'vertical', 
                        side = 'right')
x_ax = bqplot.Axis(scale = x_sc, label = "Agency Names", label_offset="50px")
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', label = "Congressional Districts", label_offset="50px")


heat_map = bqplot.GridHeatMap(color = buildings_pivot,
                              scales = {'color': color_sc,
                                        'row': y_sc,
                                        'column':x_sc },
                              interactions = {'click': 'select'}, # make interactive on click of each box
                              anchor_style = {'fill':'blue'}, # to make our selection blue
                              selected_style = {'opacity': 1.0}, # make 100% opaque if box is selected
                              unselected_style = {'opacity': 0.8}) # make a little see-through if not

# stiring and combing into 1 figure
fig_heat = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax])

####Part 2: 

x_scl = bqplot.LinearScale(min=1753,max=2019) # note: for dates on x-axis
y_scl = bqplot.LinearScale(min=500, max=5000000)
width_line = bqplot.LinearScale()

##Axes:
ax_xcl = bqplot.Axis(label='Year Acquired', scale=x_scl,label_offset="50px")
ax_ycl = bqplot.Axis(label='Total Square Footage', scale=y_scl, orientation='vertical', side='left',label_offset="50px")


###masking the plot: 

hist2d, long_edges, lat_edges = np.histogram2d(build_year,build_sqfootage, bins=[19,35])
i,j = 16,0
longs = [long_edges[i], long_edges[i+1]]
lats = [lat_edges[j],lat_edges[j+1]]
region_mask = ((build_sqfootage >= lats[0]) & (build_sqfootage<=lats[1]) &\
                (build_year >= longs[0]) & (build_year <=longs[1]) ) # picking the bins for the axis , true/false values
lats, longs, build_year_sqfootage['Square Footage'][region_mask]



### plotting line: 
Plot_lines = bqplot.Lines(x = build_year[region_mask],y = build_sqfootage[region_mask], 
                          scales={'x':x_scl, 'y':y_scl,'width': width_line}, 
                           marker_size=15, 
                           marker = 'circle', 
                           colors = ['DarkOrange'], 
                           opacities = [0.7], line_style = 'solid',
                           stroke_width = 2 )

fig_line = bqplot.Figure(marks = [Plot_lines], axes = [ax_xcl, ax_ycl])


###Interactivity: 

mySelectedLabel = ipywidgets.Label()


def on_selected(change):
    i,j = change['owner'].selected[0]
    v = buildings_values[i,j] # grab data value
    mySelectedLabel.value = 'Total Square Footge Area = ' + str(v) # set our label
    
    ### fectching axis to be changed
    longs = [long_edges[i], long_edges[i+1]]  # fetching X-axis Values for given row values
    lats = [lat_edges[j],lat_edges[j+1]] # fetching Y-axis Values for given column values
    
    ## masking region 
    region_mask = ( (build_sqfootage >= lats[0]) & (build_sqfootage<=lats[1]) ^\
                (build_year >= longs[0]) & (build_year <=longs[1]) )  # Using XOR function to increase values 
    
    ## changing values of line plot
    Plot_lines.x = build_year[region_mask] # Changing X-Axis for 
    Plot_lines.y = build_sqfootage[region_mask]  ### Changing Y-Axis for 
    # print(region_mask) 
   
heat_map.observe(on_selected, 'selected')


### Dashboard:

fig_heat.layout.min_width='500px'
fig_line.layout.min_width='500px'

ipywidgets.VBox([mySelectedLabel, ipywidgets.HBox([fig_heat,fig_line])])

### Write-up: 

### *Steps:**

    1.Grouping the Congress Disctrict & square footage to claculate total square footage values. 
    2.Create a pivot table with congress disctrict = rows and Agency Names = Coloumns.
    3.Plotted heatmap using claculated total square footage as vlaues without interactivity 
    4.Grouped "Year Acquired" to find total square footage by year as mentioned in part2
    5.Define scales, Axes to plot total square footage over "Acquired Years" using bqplot.line function 
    6.Added Interactivilty between two plots by using widgets
    7.Created a fucntion named to change the values as user clicks : On_slected
    8.Fetched the i,j vlaues on slection of the grid on heat map with change['owner'] property.
    9.Subsetting the values of rows and columns using fetched i and j 
    10.generating the region masks based of these fetched values(i, j)values by using XOR fucntion 
    11. Apply mask to the arrays created from raw dataset to see recalculated values of X and Y axis


### Q1.
Can you keep the x and y ranges static on the line plot?
Yes we can keep x and y values static by hardcoding as there is default values assigned for the lineplots.The Year and Total square footage are static.There is no interactivity in the values for X and Y axis of the Lineplot. 

The values in the above combined plot are recaliberated on the click. As the values on the gridmap are varys on the clicks, it is not possible to keep the values static on gridmap.By interlinking the two plot and making the plots intercative, the X and Y values are not static. If we click on one grid, It will Plot the true values for the mask created for respective agency and congress District.As the click on the gridmap changes, the X and Y values are reclaculate for that Click. 



### Q.2
Can you change the style? 
Yes, we can change the various factors of the plot by using bqplot features. for example,we can change the color of the grids 
by changing the opacity, Selection Color, transparency of the heat_maps and Scheme_colors for heat_maps.

For Line_plots, We can change the thickness, marks_style, the opacity and width of the lines. By using bqplot we can change
the type of the plot as well to scatter, histogram with customized features pertaining to the graph style.
bqplot build upon the grammer of graphic whoch give its a strength to vary parameters such as figure, marks, axes and scales. 
This is very systemetic way of plotting the figure with precise use of grammer of graphics.


### Q3: write-up: 
##### Any data transformations or rescalings you did for your plot
Yes, i have tranformed the data in to find the zeros with Square footage, year_acquired and Year_contructed. The scaling the x and y axis and setting time values along with width and orientation on line graph. Later masking and linking of two visulization done. 

#### How you dealt with NaN's (empty entries) in your dataset: 
The NaN were kept at the start of the dataset. During pivoting the data, we can see lot of NaN values by using nanmax 
and nanmin fuction, we get maximum value of the array( while ignoring any NaN values ) is returned. 

#### Any aesthetic choices you made (colors, layout, plot size, label size) 
and why, and what you'd like to experiment with if you had more time.

We set the orientation, colour , size, and every secondary detail. bqplot enables us to choose from various colour schemes,
achor style and type of interaction. I chose BrBG from the colorscale.By deafult it chooses, RdYIGn color. we can check the below link for more colors: 
Display all brewer palettes
https://www.datanovia.com/en/blog/the-a-z-of-rcolorbrewer-palette/ 

##### what i want to explore: 

How to modify the line plot features along with combining the heatmap like  i have plotted in the above line plot with changes in ur_Lines.colors = ['DarkOrange']
dur_Lines.opacities = [0.7]
dur_Lines.stroke_width = 2
dur_Lines.line_style = 'solid'
dur_Lines.marker = 'circle'

How to subest the raw dataset and change mask depending upon the further analysis of the data 